In [1]:
import argparse
import glob
import json
import os
import subprocess
import sys
import time

tic = time.time()
import logging as log
from distutils.spawn import find_executable
from itertools import groupby

import numpy as np

# from tensorflow.keras import models
import tensorflow as tf

# sys.path.append(os.path.dirname(os.path.abspath(__file__)))
from joblib import load

# from predictors.predictType import TypeClassification
# from annotators import prodigal_annot

In [8]:
sys.argv = [
    '_',
    '--ip',
    '/Users/fragoso/Downloads/ws/bench/files/comp/CM000950.fna.prodigal.faa.ip.tsv',
    '--prodigal',
    '/Users/fragoso/Downloads/ws/bench/files/comp/CM000950.fna.prodigal.faa',
]
__file__ = '/Users/fragoso/modules/old_emerald/emerald.py'

In [9]:
cmd = [
    "",
    "",
    "",
    "",
]

In [10]:
class tmpDir:
    """Move to directory and back to original"""

    def __init__(self, dire):
        self.dir = dire
        self.cwd = os.getcwd()

    def __enter__(self):
        os.makedirs(self.dir, exist_ok=True)
        os.chdir(self.dir)
        return self.dir

    def __exit__(self, exc_type, exc_val, exc_tb):
        os.chdir(self.cwd)


class Loss:
    """Load loss function used in trainings"""

    def __init__(self, gamma):

        self.gamma = gamma

    @tf.function
    def robustLoss(self, true, pred, sample_weight=None):
        import tensorflow.keras.backend as k

        """Akiyama, ., Imoto, K., Tonami, N., Okamoto, Y., Yamanishi, R., Fukumori, T., & Yamashita, Y. (2020).  
        Sound Event Detection Using Duration Robust Loss Function. arXiv preprint arXiv:2006.15253. 
        """
        error = (((2 - k.sigmoid(pred)) ** self.gamma) * k.log(k.sigmoid(pred))) + (
            (k.sigmoid(pred) ** self.gamma) * (1 - true) * (k.log(1 - k.sigmoid(pred)))
        )

        if sample_weight == None:
            return -k.sum(error, axis=0)
        else:
            return -k.sum(error * sample_weight, axis=0)


class AnnotationFilesToEmerald:
    """Transform external tool's files into EMERALD STR"""

    def __init__(self, shape=False):

        self.entriesDct = {}
        self.contigsDct = {}
        self.annDct = {}
        self.typeDct = {}
        self.annResults = {}
        self.gff3 = []
        self.shape = shape

    def transformIPS(self, ipsFile, f="TSV"):

        if not os.path.isfile(ipsFile):
            log.exception(f"{ipsFile} file not found")

        with open(ipsFile, "r") as h:

            if f == "GFF3":

                for l in h:

                    if l[0] == "#":
                        continue

                    if "protein_match" in l:
                        spl2 = l.split("\t")[-1][:-2]
                        eDct = dict([x.split("=") for x in spl2.split(";")])
                        entry = (
                            eDct["Dbxref"].split(":")[-1][:-1]
                            if "Dbxref" in eDct
                            else eDct["Name"]
                        )
                        self.entriesDct.setdefault(
                            eDct["Target"].split()[0], []
                        ).append(entry)

            elif f == "TSV":

                for l in h:

                    spl = l.split("\t")
                    self.entriesDct.setdefault(spl[0], []).append(
                        spl[11] if (len(spl) > 11 and spl[11][:2] == "IP") else spl[4]
                    )
            else:
                log.exception(f"Unrecognised IPS format")

    def transformEmeraldHmm(self, hmmFiles):

        for hmmFile in hmmFiles:
            log.info(f"processing {hmmFile}")
            if not os.path.isfile(hmmFile):
                log.exception(f"{hmmFile} file not found")

            with open(hmmFile, "r") as h:

                for l in h:

                    if l[0] == "#":
                        continue

                    spl = l.split()
                    self.entriesDct.setdefault(spl[3], []).append(spl[0])

    def transformCDSpredToCDScontigs(self, cdsPredFile, f="FASTA"):

        if not os.path.isfile(cdsPredFile):
            log.exception(f"{cdsPredFile} file not found")

        with open(cdsPredFile, "r") as h:

            if f == "FASTA":

                for l in h:

                    if l[0] != ">":
                        continue

                    spl = l.split()
                    start, end = int(spl[2]), int(spl[4])

                    self.contigsDct.setdefault(
                        "_".join(spl[0].split("_")[:-1])[1:], []
                    ).append((spl[0][1:], (start, end)))

            elif f == "GBK":

                from Bio import SeqIO

                recs = list(SeqIO.parse(open(cdsPredFile, "r"), "gb"))

                for rec in recs:
                    for f in rec.features:
                        if f.type == "CDS":

                            start, end = int(f.location.start) + 1, int(f.location.end)

                            self.contigsDct.setdefault(rec.id, []).append(
                                (
                                    f.qualifiers["protein_id"]
                                    if "protein_id" in f.qualifiers
                                    else f.qualifiers["locus_tag"],
                                    (start, end),
                                )
                            )

    def buildMatrices(self, annVocab):

        log.info("Building matrices")

        for contig in self.contigsDct:

            cdss = self.contigsDct[contig]

            samps = (
                len(cdss)
                if not self.shape
                else ((len(cdss) // self.shape) + 1) * self.shape
            )
            log.info(f"self.samps {samps}")

            self.annDct[contig] = np.zeros((samps, len(annVocab)))
            #             self.typeDct[contig] = np.zeros((samps, len(typeVocab)))

            for ix, cds in enumerate(cdss):

                name = cds[0]
                if name not in self.entriesDct:
                    continue

                modiAnn = [annVocab[x] for x in self.entriesDct[name] if x in annVocab]
                #                 modiType = [
                #                     typeVocab[x] for x in self.entriesDct[name] if x in typeVocab
                #                 ]
                self.annDct[contig][ix][modiAnn] = 1

    #                 self.typeDct[contig][ix][modiType] = 1

    def predictAnn(self, model, colapseFunc=max):

        log.info("Predict BGC probability w/ TensorFlow")
        for contig in self.annDct:
            mat = self.annDct[contig]
            xva_, vaIx = self.transformMat(mat)
            vaS = (xva_.shape[0] // self.shape) * self.shape
            xva1 = xva_[:vaS].reshape(vaS // self.shape, self.shape)
            xva = np.append(
                xva1,
                [list(xva_[vaS:]) + [0] * (self.shape - (vaIx.shape[0] - vaS))],
                axis=0,
            )
            self.predict_ = model.predict(xva)
            predict = self.predict_.reshape(vaS + self.shape)
            predict_str_, nix = self.partialReStrMat(vaIx, predict, func=colapseFunc)
            self.annResults[contig] = self.projectRes(predict_str_, nix, mat.shape[0])

    def transformMat(self, mat):

        scuash = np.array(list(map(lambda x: np.where(x == 1)[0], mat)))
        nli, nix = [], []
        for ix, x in enumerate(scuash):
            if x.shape[0] == 0:
                continue
            nli.extend(x)
            nix.extend([ix] * x.shape[0])
        sups = [
            (k, list(zip(*g))[1][0])
            for k, g in groupby(zip(nli, nix), key=lambda x: x[0])
        ]
        if len(nli) == 0:
            return np.array([0]), np.array([0])
        a, b = list(zip(*sups))
        a, b = np.array(a), np.array(b)
        return a, b

    def projectRes(self, res, ixes, lenOri):

        nres_ = np.zeros(lenOri)
        nres_[ixes] = res
        prev = 0
        rprev = 0
        for ix, r in zip(ixes, res):
            if ix - 1 == prev:
                prev = ix
                rprev = r
                continue
            for ixx, s in list(enumerate(np.linspace(rprev, r, ix - prev + 1)))[1:-1]:
                nres_[prev + ixx] = s
            prev = ix

        return nres_

    def partialReStrMat(self, Ix, res, func):

        nres, nix = [], []
        for k, g in groupby(zip(res, Ix), key=lambda x: x[1]):

            gg = list(list(zip(*g))[0])
            nix.append(k)
            nres.append(func(gg))
        return np.array(nres), np.array(nix)

    def defineLooseClusters(self, thBase=0.875, thBorder=0.9, rmless=3, fill=3):

        log.info("Define Clusters")
        self.bridged, self.looseClst, self.borderClst, self.typesClst = {}, {}, {}, {}

        for contig in self.annResults:
            self.looseClst[contig] = np.array(
                self.rmLessThan(
                    self.fillGap(
                        np.where(self.annResults[contig] < thBase, 0, 1), fill
                    ),
                    rmless,
                )
            )
            self.borderClst[contig] = np.where(self.annResults[contig] < thBorder, 0, 1)
            self.typesClst[contig] = np.zeros((len(self.annResults[contig]), 8))

    def rmLessThan(self, contig, n):
        newContig = []
        for k, g in groupby(contig):
            if k == 0:
                newContig.extend(list(g))
            else:
                p = list(g)
                if len(p) <= n:
                    newContig.extend([0 for x in p])
                else:
                    newContig.extend(p)
        return newContig

    def fillGap(self, contig, gap):
        newContig = []
        for k, g in groupby(contig):
            gg = list(g)
            if k == 0 and len(gg) <= gap:
                newContig.extend([1 for x in gg])
            else:
                newContig.extend(gg)
        return newContig

    def predictType(self, typeModel, allTh, posTh, g):

        log.info("Predict BGC classes")
        greed = {0:0.58, 1: 0.41, 2: 0.345, 3: 0}

        #         posTh = greed[g] if g != None else posTh
        posTh = greed[g] if posTh == None else posTh
        log.info(f"Pos_class model Thres {posTh}")
        locations, matrix, typeLi = [], [], []
        claa = [
                    "Alkaloid",
                    "All",
                    "NRP",
                    "Polyketide",
                    "RiPP",
                    "Saccharide",
                    "Terpene",
                    "Other",
                ]

        for contig in self.contigsDct:
            for k, g in groupby(enumerate(self.looseClst[contig]), key=lambda x: x[1]):
                if k == 0:
                    continue
                gg = list(list(zip(*g))[0])
                tmat_ = np.sum(self.annDct[contig][gg], axis=0)
                tmat = np.where(tmat_ > 0, 1, 0)
                #                 typePrediction.setdefault(contig)
                locations.append((contig, gg))
                matrix.append(tmat)
        if len(matrix) > 0:
            pred = np.empty((len(matrix), len(claa)))
            for nc, cla in enumerate( claa ):
                pred[:, nc] = typeModel[cla].predict_proba(matrix)[:, 1]
        else:
            pred = []

        for ix, p in enumerate(pred):

            contig, gg = locations[ix]

            if not (p[1] >= allTh and np.max(p[[0, 2, 3, 4, 5, 6, 7]]) >= posTh):
            #if not (p[1] >= allTh and np.max(p[[0, 4, 5, 6]]) >= posTh) and not (p[1] >= allTh and np.max(p[[2, 3]]) >= posTh-0.07):
                self.looseClst[contig][gg] = 0
                self.borderClst[contig][gg] = 0

            else:
                log.info(str(p))
                self.typesClst[contig][gg] = p

    def writeGff3(self, outfile="emerald.full.gff", emrldV=0.1):

        log.info("build GFF3")
        log.info(outfile)
        type_code = {
            0: "Alkaloid",
            1: "All",
            2: "NRP",
            3: "Polyketide",
            4: "RiPP",
            5: "Saccharide",
            6: "Terpene",
            7: "Other",
        }

        for contig in self.looseClst:

            ### annot cds
            for ix, f in enumerate(self.contigsDct[contig]):

                ID, (start, end) = f
                emrldProb = "{:.3f}".format(self.annResults[contig][ix])
                self.gff3.append(
                    f"{contig}\tEMERALDv{emrldV}\tCDS\t{start}\t{end}\t.\t.\t.\tID={ID};emerald_probability={emrldProb}"
                )

            ### annot clusters

            ct = 1
            for k, g in groupby(
                enumerate(self.looseClst[contig][: len(self.contigsDct[contig])]),
                key=lambda x: x[1],
            ):

                if k == 0:
                    continue

                ID = f"{contig}_emrld_{ct}"
                ct += 1

                gg = list(list(zip(*g))[0])

                start, end = (
                    self.contigsDct[contig][gg[0]][1][0],
                    self.contigsDct[contig][gg[-1]][1][1],
                )

                ### asses if bgc in edge of contig
                edge = (
                    str(True)
                    if (gg[0] == 0 or gg[-1] == len(self.contigsDct[contig]) - 1)
                    else str(False)
                )

                tClst = self.typesClst[contig][gg[0]]
                typs = ",".join(
                    [
                        "{}:{:.3f}".format(type_code[ix], tClst[ix])
                        for ix in [0, 2, 3, 4, 5, 6, 7]
                    ]
                )

                self.gff3.append(
                    f"{contig}\tEMERALDv{emrldV}\tCLUSTER\t{start}\t{end}\t.\t.\t.\tID={ID};Class_Probability={typs};Edge={edge}"
                )

                ct2 = 1
                for k2, g2 in groupby(
                    zip(gg, self.borderClst[contig][gg]), key=lambda x: x[1]
                ):
                    if k2 == 0:
                        continue
                    gg2 = list(list(zip(*g2))[0])
                    start, end = (
                        self.contigsDct[contig][gg2[0]][1][0],
                        self.contigsDct[contig][gg2[-1]][1][1],
                    )
                    ### asses if bgc in edge of contig
                    edge = (
                        str(True)
                        if (gg2[0] == 0 or gg2[-1] == len(self.contigsDct[contig]) - 1)
                        else str(False)
                    )

                    self.gff3.append(
                        f"{contig}\tEMERALDv{emrldV}\tCLUSTER_border\t{start}\t{end}\t.\t.\t.\tID={ID}_{ct2};Class_Probability={typs};Edge={edge}"
                    )
                    ct2 += 1

        log.info(f"Writing output to file {outfile}")
        with open(outfile, "w") as h:

            h.write("##gff-version 3\n")

            for l in sorted(
                self.gff3,
                key=lambda x: (
                    x.split("\t")[0],
                    int(x.split("\t")[3]),
                    x.split("\t")[2],
                ),
            ):

                h.write(f"{l}\n")


class Preprocess:
    """External tools needed for emerald bgc detection"""

    def __init__(self, cpus, outdir=None):

        self.cpus = int(cpus)
        self.outdir = outdir if outdir else "temp"

    def runProdigal(self, fna, meta=False):
        """Predict genes using prodigal"""
        log.info("Progial gene prediction...")

        if not find_executable("prodigal"):
            log.exception("Parodigal is not installed or in PATH")

        if not os.path.isfile(fna):
            log.exception(f"{fna} file not found")

        outFaa = "{}.prodigal.faa".format(os.path.basename(fna))
        log.info(f"{fna} {outFaa}")
        cmd = ["prodigal", "-i", fna, "-a", outFaa] + (["-p", "meta"] if meta else [])

        with tmpDir(self.outdir):
            #             try:
            outs, errs = subprocess.Popen(
                cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE
            ).communicate()
            if "Error:  Sequence must be 20000 characters" in errs.decode("utf8"):

                log.info(
                    "Sequence must be 20000 characters when running Prodigal in normal mode. Trying -p meta"
                )

                cmd = ["prodigal", "-i", fna, "-a", outFaa] + (["-p", "meta"])

                outs, errs = subprocess.Popen(
                    cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE
                ).communicate()

            #                 log.info(outs)
            log.info(errs.decode("utf8"))
            #                 log.info("Prodigal succesful")
            #             except subprocess.CalledProcessError as err:
            #                 print("Failed")
            #                 log.exception(err.output)

            """ Remove asterixs from faa
                """
            log.info("Removing asterix from prodigal faa")
            with open(outFaa, "r") as h:
                noAstFaa = [x.replace("*", "") for x in h]
            with open(outFaa, "w") as h:
                for l in noAstFaa:
                    h.write(f"{l}")

            return os.path.abspath(outFaa)

    def gbkToProdigal(self, gbk):
        """Transform gbk to faa. This enables preprocessing with sequences"""
        log.info("write gbk as faa")
        from Bio import SeqIO

        recs = list(SeqIO.parse(open(gbk, "r"), "gb"))

        base = os.path.basename(gbk)

        with tmpDir(self.outdir):

            with open(f"{base}.faa", "w") as h:

                for rec in recs:

                    for f in rec.features:

                        if f.type == "CDS" and "translation" in f.qualifiers:

                            pid = (
                                f.qualifiers["protein_id"]
                                if "protein_id" in f.qualifiers
                                else f.qualifiers["locus_tag"]
                            )
                            seq = f.qualifiers["translation"][0]
                            h.write(f">{pid}\n{seq}\n")

            return os.path.abspath(f"{base}.emerald.faa")

    def runHmmScan(self, faa, hmmLib):
        """annotate functionally with emerald hmm library and hmmScan"""
        log.info("emerald functional annotation...")

        if not find_executable("hmmscan"):
            log.exception("hmmscan is not installed or in PATH")

        if not os.path.isfile(faa):
            log.exception(f"{faa} file not found")

        if not os.path.isfile(hmmLib):
            log.exception(f"{hmmLib} file not found")

        outTsv = "{}.emerald.tsv".format(os.path.basename(faa))

        cmd = (
            ["hmmscan", "--domtblout", outTsv, "--cut_ga"]
            + (["--cpu", str(self.cpus)] if self.cpus else [])
            + ([hmmLib, faa])
        )
        log.info(cmd)
        with tmpDir(self.outdir):
            try:
                #                 subprocess.check_call(
                outs, errs = subprocess.Popen(
                    cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE
                ).communicate()
                log.info(errs.decode("utf8"))
            #                 log.info(outs)
            #                 log.info("emerald annotation succesful")
            except subprocess.CalledProcessError as err:
                log.exception(err.output)

            return os.path.abspath(outTsv)

    def runInterproscan(
        self,
        faa,
        analyses=["Pfam", "TIGRFAM", "PRINTS", "ProSitePatterns", "Gene3D"],
    ):
        """annotate functionally with InterproScan"""
        log.info(
            "InterProScan functional annotation... version 83 is user responsability"
        )

        if not find_executable("interproscan.sh"):
            log.exception("interproscan.sh is not installed or in PATH")

        if not os.path.isfile(faa):
            log.exception(f"{faa} file not found")

        outGff = "{}.ip.tsv".format(os.path.basename(faa))
        cmd = (
            ["interproscan.sh", "-i", faa, "-o", outGff, "-f", "TSV"]
            + (["-appl", ",".join(analyses)] if analyses else [])
            + (["-cpu", str(self.cpus)] if self.cpus else [])
        )
        log.info(" ".join(cmd))

        with tmpDir(self.outdir):
            #             try:
            outs, errs = subprocess.Popen(
                cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE
            ).communicate()
            log.info(outs.decode("utf8"))
            log.info(errs.decode("utf8"))
            #                 log.info("Prodigal succesful")
            #             except subprocess.CalledProcessError as err:
            #                 log.exception(err.output)

            return os.path.abspath(outGff)

In [11]:
parser = argparse.ArgumentParser(
    description="EMERALD. detect SM Biosynthetic Gene Clusters"
)
parser.add_argument(
    "--fna",
    dest="fna",
    type=str,
    default=False,
    help="nucleotide sequnce file, this option overrides -gbk, -ip,-ih",
    metavar="FILE",
)
# parser.add_argument( "--prod", dest="prodigal",
#                  help="prodigal translation sequnce file", metavar="FILE")
parser.add_argument(
    "--gbk",
    dest="gbk",
    type=str,
    default=False,
    help="protein annotated genebank file",
    metavar="FILE",
)
parser.add_argument(
    "--prodigal",
    dest="prodigal",
    type=str,
    help="tranlations from prodigal file",
    metavar="FILE",
)
parser.add_argument(
    "--ip", dest="interpro", type=str, help="InterProScan v83 file", metavar="FILE"
)
parser.add_argument(
    "--ipf",
    dest="interpro_f",
    type=str,
    help="InterProScan file format. TSV,GFF3... GFF3 not supported",
    default="TSV",
    metavar="FILE",
)
parser.add_argument(
    "--ih",
    dest="inhouse",
    type=str,
    help="inHouse TSV file",
    metavar="FILE",
    nargs="+",
)
parser.add_argument(
    "--thres",
    dest="thBase",
    default=0.87333,
    type=float,
    help="BGC probability threshold",
    metavar="FLOAT",
)
parser.add_argument(
    "--thBorder",
    dest="thBorder",
    default=0.98,
    type=float,
    help="BGC border refinement threshold",
    metavar="FLOAT",
)
parser.add_argument(
    "--thPos",
    dest="posTh",
    default=None,
    type=float,
    help="postProcess negative filter threshold",
    metavar="FLOAT",
)
parser.add_argument(
    "--thFilter",
    dest="thFilter",
    default=0,
    type=float,
    help="postProcess negative filter threshold",
    metavar="FLOAT",
)
parser.add_argument(
    "--greed",
    dest="greed",
    default=2,
    type=int,
    help="Level of greediness. 0,1,2,3",
    metavar="INT",
)
parser.add_argument(
    "--meta",
    dest="meta",
    default=False,
    type=bool,
    help="prodigal option meta [default False]",
    metavar="BOOL",
)
parser.add_argument(
    "--outdir",
    dest="outdir",
    type=str,
    help="output directory. default $PWD",
    metavar="DIRECTORY",
)
parser.add_argument(
    "--outfile",
    dest="outfile",
    type=str,
    help="output file, overrides --outdir",
    metavar="DIRECTORY",
)
parser.add_argument(
    "--cpu",
    dest="cpu",
    default=1,
    type=int,
    help="cpus for INTERPROSCAN and HMMSCAN",
    metavar="INT",
)
#     parser.add_argument(
#         "--clasModFiles",
#         dest="clasModFiles",
#         type=str,
#         help="altenative ann model",
#         metavar="STR",
#     )
parser.add_argument(
    "--clasModFiles",
    dest="clasModFiles",
    type=str,
    help="alternative class models",
    metavar="FILE",
    nargs="+",
)
parser.add_argument(
    "--model",
    dest="model",
    type=str,
    help="altenative ann model",
    metavar="STR",
)
parser.add_argument(
    "--vocab",
    dest="vocab",
    type=str,
    help="alternative ann vocab",
    metavar="STR",
)
parser.add_argument(
    "--shape",
    dest="shape",
    type=int,
    default=200,
    help="window size for altenative ann model",
    metavar="INT",
)
parser.add_argument(
    "--rmless",
    dest="rmless",
    type=int,
    default=1,
    help="minimum bgc length",
    metavar="INT",
)
parser.add_argument(
    "--fill",
    dest="fill",
    type=int,
    default=3,
    help="fill gap between dtected regions",
    metavar="INT",
)

args = parser.parse_args()
#     return args
basef = (
    args.gbk
    if args.gbk
    else args.fna
    if args.fna
    else args.prodigal
    if args.prodigal
    else None
)
base = os.path.basename(basef)

outdir = (
    os.path.abspath(args.outdir) if args.outdir else os.path.join(os.getcwd(), f'{base}.emrld')
    )
with tmpDir(outdir):
    log.basicConfig(
        filename=f"{outdir}/emerald.log",
        level=log.DEBUG,
        format="%(asctime)s %(message)s",
    )
    print(f"LOG_FILE: {outdir}/emerald.log")
    log.info("************     *******************\n   EMRELAD...\n  ************")
    log.info(f"outdir: {outdir}")

log.info("loading models")
hmmlib = os.path.join(
    os.path.dirname(os.path.abspath(__file__)), "models", "emerald.hmm"
)
model_BGC_file = os.path.join(
    os.path.dirname(os.path.abspath(__file__)),
    "models",
    "emerald.h5" if not args.model else args.model,
)
log.info(f"model file: {model_BGC_file}")
model_class_files = os.path.join(
    os.path.dirname(os.path.abspath(__file__)), "models", "*.rf.clf.joblib"
)  # if not args.clasModFiles else args.clasModFiles
annVocab_file = os.path.join(
    os.path.dirname(os.path.abspath(__file__)),
    "models",
    "vocab.json" if not args.vocab else args.vocab,
)
#     typeVocab_file = os.path.join(
#         os.path.dirname(os.path.abspath(__file__)), "models", "typeVocab.json"
#     )

with open(annVocab_file, "r") as h:
    annVocab = json.load(h)

#     with open(typeVocab_file, "r") as h:
#         typeVocab = json.load(h)
clfClass = {}
for f in (
    glob.glob(model_class_files) if not args.clasModFiles else args.clasModFiles
):
    cla = os.path.basename(f).split(".")[0]
    clfClass[cla] = load(f)

rlf = Loss(None)
annModel = tf.keras.models.load_model(
    model_BGC_file, custom_objects={"robustLoss": rlf.robustLoss}
)

log.info("preprocessing files")
preprocess = Preprocess(cpus=args.cpu, outdir=outdir)

log.info("asses input format")
if args.gbk:
    log.info("process gbk file")
    prodigal_file = preprocess.gbkToProdigal(os.path.abspath(args.gbk))
    cds_f = "GBK"

elif args.fna or args.prodigal:
    log.info("get prodigal file")
    prodigal_file = (
        preprocess.runProdigal(os.path.abspath(args.fna), meta=args.meta)
        if args.fna
        else args.prodigal
        if args.prodigal
        else None
    )
    cds_f = "FASTA"
#         file = args.fna if args.fna else args.prodigal

else:
    log.critical("No input file provides. --fna,--gbk,--prodigal. check -help")

log.info("get inhouse hmm file")
hmm_files = (
    args.inhouse
    if args.inhouse
    else [preprocess.runHmmScan(prodigal_file, hmmlib)]
    if prodigal_file
    else None
)

log.info("get interproscan file")
ips_format, ips_file = (
    (args.interpro_f, args.interpro)
    if (args.interpro_f and args.interpro)
    else ("TSV", preprocess.runInterproscan(prodigal_file))
    if prodigal_file
    else None
)

log.info("EMERALD process")
annotate = AnnotationFilesToEmerald(shape=args.shape)

log.info("transform interpro file")
annotate.transformIPS(ips_file, f=ips_format)

log.info("transform inhouse hmm file")
annotate.transformEmeraldHmm(hmm_files)

log.info("transform cds file")
annotate.transformCDSpredToCDScontigs(prodigal_file, f=cds_f)

log.info("transform dicts to np matrices")
annotate.buildMatrices(annVocab)

log.info("predict bgc regions")
annotate.predictAnn(annModel)

log.info("define clusters")
annotate.defineLooseClusters(
    thBase=args.thBase, thBorder=args.thBorder, rmless=args.rmless, fill=args.fill
)

log.info("post-processing filters and type classification")
annotate.predictType(clfClass, allTh=args.thFilter, posTh=args.posTh, g=args.greed)

In [14]:
log.info("write gff file")
annotate.writeGff3(
    outfile=args.outfile if args.outfile else f"{outdir}/{base}.emerald.full.gff2",
    emrldV=0.1,
)

# log.info("EMERALD succesful")
# run_time = tic - time.time()
# log.info(f"RUNNING TIME: {run_time}")
print("EMERALD succesful")

EMERALD succesful


In [15]:
f"{outdir}/{base}.emerald.full.gff2"

'/Users/fragoso/modules/emeraldbgc/emeraldbgc/modules/CM000950.fna.prodigal.faa.emrld/CM000950.fna.prodigal.faa.emerald.full.gff2'

In [16]:
!grep "TER\s" /Users/fragoso/modules/emeraldbgc/emeraldbgc/modules/CM000950.fna.prodigal.faa.emrld/CM000950.fna.prodigal.faa.emerald.full.gff2

CM000950	EMERALDv0.1	CLUSTER	235657	280316	.	.	.	ID=CM000950_emrld_1;Class_Probability=Alkaloid:0.020,NRP:0.080,Polyketide:0.400,RiPP:0.070,Saccharide:0.120,Terpene:0.210,Other:0.240;Edge=False
CM000950	EMERALDv0.1	CLUSTER	651667	716832	.	.	.	ID=CM000950_emrld_2;Class_Probability=Alkaloid:0.410,NRP:0.880,Polyketide:0.720,RiPP:0.490,Saccharide:0.450,Terpene:0.270,Other:0.570;Edge=False
CM000950	EMERALDv0.1	CLUSTER	747262	777103	.	.	.	ID=CM000950_emrld_3;Class_Probability=Alkaloid:0.030,NRP:0.050,Polyketide:0.050,RiPP:0.030,Saccharide:0.260,Terpene:0.440,Other:0.120;Edge=False
CM000950	EMERALDv0.1	CLUSTER	917061	974020	.	.	.	ID=CM000950_emrld_4;Class_Probability=Alkaloid:0.070,NRP:0.110,Polyketide:0.200,RiPP:0.420,Saccharide:0.050,Terpene:0.020,Other:0.170;Edge=False
CM000950	EMERALDv0.1	CLUSTER	1451219	1472608	.	.	.	ID=CM000950_emrld_5;Class_Probability=Alkaloid:0.000,NRP:0.010,Polyketide:0.000,RiPP:0.400,Saccharide:0.000,Terpene:0.000,Other:0.020;Edge=False
CM000950	EMERALDv0.1	CLUSTER

In [13]:
c = 'CM000950'

for k,g in groupby(enumerate(annotate.looseClst[c]),key = lambda z:z[1]):
    if k ==0:
        continue
    gg = list(zip(*g))
    print(gg[0][0],gg[0][-1])

218 250
587 629
657 684
810 867
1280 1297
1553 1605
1643 1690
1757 1777
2081 2086
2552 2570
3169 3198
3316 3345
3453 3469
3962 3990
4139 4161
4439 4452
4537 4538
4594 4608
4702 4712
4745 4755
5315 5340
5655 5672
5884 5926
6527 6548
6672 6791
6800 6918
